In [2]:
from keras import models
from keras import layers
import keras
from csv import reader
from keras.utils import to_categorical
import numpy


Using TensorFlow backend.


The next block of code is the one that is in charge of cleaning, building, spliting and scaling the ds

In [0]:
def load_csv(filename):
  """ This function cleans the dataset from the rows that have Null and retur a clean dataset"""
  dataset = []
  with open(filename, 'r') as file:
    csvReader =  reader(file)
    for row in csvReader:
      if (not row) or ("NA" in row):
        continue
      dataset.append(row[3:])
  dataset.pop(0)#This is to skip the names of the columns
  return dataset


def str_column_to_float(dataset):#This changes all the values to float values, to avoid problems with max function
  for i in range(len(dataset[0])):
    for row in dataset:
      row[i] = float(row[i].strip())

def scale(dataset): #this gives us values between -1 and 1
  minmax = []
  for i in range(len(dataset[0])): #This is looking in each column, row example:['0', '39', '4', '1', '9', '0', '0', '0', '0', '170', '110.5', '69', '22.19', '60', '103', '0']
    columnValues = [row[i] for row in dataset]
    acum = 0
    for row in columnValues:
      acum += row
    value_min = min(columnValues)
    value_max = max(columnValues)
    minmax.append([value_min, value_max])
  for row in dataset:
    for i in range(len(row)):
      if(minmax[i][0] != 1 and (minmax[i][1] - minmax[i][0]) != 0 ):
         row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0]) 
  

def getYX(ds):
  y = []
  for row in ds:
    temp = row.pop(len(row)-1)
    y.append(temp)
  return y

def splitDs (dataset,splits):
  splitData = []
  cpyDs = list(dataset)
  splitSize = int(len(dataset)/splits)
  for i in range(splits):
    split= []
    for j in range(splitSize):
      split.append(cpyDs.pop())
    splitData.append(split)
  return splitData

def getDs(filename): #Build the complete and clean ds
  ds = load_csv(filename)
  str_column_to_float(ds)
  scale(ds)
  return splitDs(ds,2)

In [5]:
network = models.Sequential()
network.add(layers.Dense(128,activation='tanh', input_shape=(36,)))
network.add(layers.Dense(64,activation='tanh'))
network.add(layers.Dense(32,activation='tanh'))
network.add(layers.Dense(16,activation='tanh'))
network.add(layers.Dense(4,activation='softmax'))



ds = getDs("CTG.csv")

dsTrainY = getYX(ds[0])
dsTrainY = numpy.asarray(dsTrainY).astype('float32')
#dsTrainY = to_categorical(dsTrainY)
print("Train Label Shape", dsTrainY.shape)

dsTrainX = ds[0]
dsTrainX = numpy.array(dsTrainX)
print("Train In Shape",dsTrainX.shape)


dsTestY = getYX(ds[1])
dsTestY = numpy.asarray(dsTestY).astype('float32')
#dsTestY = to_categorical(dsTestY)
print("Test Labels Shape", dsTestY.shape)


dsTestX = ds[1]
dsTestX = numpy.array(dsTestX)
print("Test In Shape",dsTestX.shape)


Train Label Shape (1063,)
Train In Shape (1063, 36)
Test Labels Shape (1063,)
Test In Shape (1063, 36)


In [6]:
network.summary()
opti = keras.optimizers.Adam(learning_rate=0.0001)
network.compile(optimizer=opti,
		loss='sparse_categorical_crossentropy',
		metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               4736      
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 68        
Total params: 15,668
Trainable params: 15,668
Non-trainable params: 0
_________________________________________________________________


In [7]:
network.fit(dsTrainX,dsTrainY,validation_data=(dsTestX,dsTestY),epochs = 25)

test_loss, test_acc = network.evaluate(dsTestX, dsTestY)
print ('test_acc:', test_acc, ' test_loss:', test_loss)

Train on 1063 samples, validate on 1063 samples
Epoch 1/25
1063/1063 [==============================] - 1s 497us/step - loss: 1.6250 - accuracy: 0.1552 - val_loss: 1.1684 - val_accuracy: 0.5118
Epoch 2/25
1063/1063 [==============================] - 0s 104us/step - loss: 0.8316 - accuracy: 0.8627 - val_loss: 0.9067 - val_accuracy: 0.6905
Epoch 3/25
1063/1063 [==============================] - 0s 93us/step - loss: 0.5676 - accuracy: 0.8664 - val_loss: 0.9085 - val_accuracy: 0.6905
Epoch 4/25
1063/1063 [==============================] - 0s 111us/step - loss: 0.4966 - accuracy: 0.8664 - val_loss: 0.8900 - val_accuracy: 0.6905
Epoch 5/25
1063/1063 [==============================] - 0s 94us/step - loss: 0.4514 - accuracy: 0.8664 - val_loss: 0.8408 - val_accuracy: 0.6905
Epoch 6/25
1063/1063 [==============================] - 0s 92us/step - loss: 0.4103 - accuracy: 0.8664 - val_loss: 0.8098 - val_accuracy: 0.6905
Epoch 7/25
1063/1063 [==============================] - 0s 94us/step - loss: 0.

In [0]:
for i in range(1000):
  if(dsTestY[i] == 3.0):
    print(i)

118
238
239
289
291
292
293
294
295
311
312
313
342
356
358
359
360
361
362
364
365
372
379
380
408
409
410
411
412
417
418
465
466
587
588
589
590
591
592
593
594
595
596
644
659
698
699
710
711
720
721
722
723
727
728
734
738
739
742
743
745
753
764
765
766
822
946
968
969


In [0]:
arr = [dsTestX[118]]
arr = numpy.array(arr)
auxArr = network.predict(arr)
index0=numpy.argmax(auxArr[0])
print(index0)

# index1=numpy.argmax(auxArr[1])
#index2=numpy.argmax(auxArr[2])


# print(index0,index1,index2)


3
